## 📌 문장 단위로 복원한 리뷰 다시 합치기

- [복원된 리뷰를 바탕을 난독화된 리뷰 문장 단위로 분할](./split_test_sentences.ipynb)한 이후에 분할한 문장을 모델에게 입력하여 문장 단위로 복원합니다.
- 복원한 이후에 다시 원래와 같은 리뷰의 형태로 만드는 작업을 진행해야 합니다.
- 이때, 문장 단위로 복원한 데이터([submission_sentences.csv](../submissions/submission_sentences.csv))에 해당 문장이 전체 리뷰에서 어느 위치에 있는지를 기록하였기 때문에 이를 바탕으로 전체 리뷰에서 해당 문장에 대응되는 부분을 교체해줍니다.
- 문장 단위로 리뷰를 복원하였음에도 글자 수가 일치하지 않는 부분에 대해서는 전체 리뷰 단위로 복원한 이후에 후처리한 데이터([submission_postprocess.csv](../submissions/submission_postprocess.csv))를 사용하여 글자 수가 기존 테스트 데이터와 일치할 수 있도록 합니다.
- 최종적으로 이러한 작업을 거치게 되면 테스트 데이터에 있는 난독화된 리뷰를 복원한 데이터([submission_final.csv](../submissions/submission_final.csv))이 저장됩니다.

In [1]:
import pandas as pd

sub_sentences = pd.read_csv('../submissions/submission_sentences.csv', encoding='utf-8-sig') # 문장 단위로 복원된 리뷰
post_df = pd.read_csv('../submissions/submission_postprocess.csv', encoding='utf-8-sig') # 학습된 모델이 복원된 리뷰를 후처리한 리뷰 (난독화된 리뷰와 길이 일치)

In [2]:
sub_sentences.head()

,ID,input,start_idx,end_idx,output
0,TEST_0000,녀뮨넒뭅 만죡숭러윤 효템뤼에오.,0,17,너무너무 만족스러운 호텔이에요.
1,TEST_0000,푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오.,18,40,부산에 오면 꼭 추천하고 싶은 곳이네요.
2,TEST_0000,췌꾜윕뉘댜! ㅎㅎ,41,50,최고입니다! ㅎㅎ
3,TEST_0000,당음웨 또 옭 컷 갗았요.,51,65,다음에 또 올 것 같아요.
4,TEST_0001,"풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰...",0,58,"프론트가 없고, 조식도 없으며, 일반 입주민들이 사이트임에 있어 호텔처럼 관리가 잘..."


In [3]:
post_df.head()

,ID,output
0,TEST_0000,너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다...
1,TEST_0001,"프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 잘..."
2,TEST_0002,진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원인지 사장인지 체크...
3,TEST_0003,뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에...
4,TEST_0004,방 상태는 진짜 폐허 직전인데 전망은 좋아요. 보일러가 아주 찬찬하게 돌아서 추웠어...


In [4]:
from kss import split_sentences

post_df['sentences'] = post_df["output"].apply(lambda x: split_sentences(x)) # 재복원하였음에도 글자 수가 일치하지 않는 문제를 해결하기 위해 1차 후처리된 리뷰를 문장 단위로 분리

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [5]:
post_df['sentences'].head()

0    [너무너무 만족스러운 호텔이에요., 부산에 오면 꼭 추천하고 싶은 곳이에요., 최고...
1    [프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 ...
2    [진짜 불친절해요., 살면서 머물렀던 호텔 중에 최악이었습니다., 직원인지 사장인지...
3    [뷰 맛집~~ 그런데 방음이 미흡하네요., 층간 소음과 발코니가 이중창이 아니라서 ...
4    [방 상태는 진짜 폐허 직전인데 전망은 좋아요., 보일러가 아주 찬찬하게 돌아서 추...
Name: sentences, dtype: object

In [6]:
post_sentences = {"ID": [], "output": []}
for _, row in post_df.iterrows():
    for sent in row['sentences']:
        post_sentences["ID"].append(row["ID"])
        post_sentences["output"].append(sent)

post_sentences_df = pd.DataFrame(post_sentences)
post_sentences_df.head()

,ID,output
0,TEST_0000,너무너무 만족스러운 호텔이에요.
1,TEST_0000,부산에 오면 꼭 추천하고 싶은 곳이에요.
2,TEST_0000,최고입니다! ㅎㅎ
3,TEST_0000,다음에 또 올 것 같아요.
4,TEST_0001,"프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 잘..."


In [7]:
import re

# 단어와 공백을 분리하는 함수
def split_with_spaces(text):
    # 공백(1개 이상)과 단어를 분리
    return re.split(r'(\s+)', text)

sub_sentences['input_words'] = sub_sentences['input'].apply(split_with_spaces)
sub_sentences['output_words'] = sub_sentences['output'].apply(split_with_spaces)
post_sentences_df['post_words'] = post_sentences_df['output'].apply(split_with_spaces)


In [8]:
sub_sentences['input_words'].head()

0                          [녀뮨넒뭅,  , 만죡숭러윤,  , 효템뤼에오.]
1         [푸싸눼,  , 옰면,  , 콕,  , 츄쩐학꼬,  , 싶은,  , 콧쉰웨오.]
2                                      [췌꾜윕뉘댜!,  , ㅎㅎ]
3                     [당음웨,  , 또,  , 옭,  , 컷,  , 갗았요.]
4    [풀룐투갸,  , 엎코,,  , 좀식또,  , 업읍머,,  , 윌뱐,  , 잎츔민든...
Name: input_words, dtype: object

In [9]:
sub_sentences['output_words'].head()

0                          [너무너무,  , 만족스러운,  , 호텔이에요.]
1         [부산에,  , 오면,  , 꼭,  , 추천하고,  , 싶은,  , 곳이네요.]
2                                      [최고입니다!,  , ㅎㅎ]
3                     [다음에,  , 또,  , 올,  , 것,  , 같아요.]
4    [프론트가,  , 없고,,  , 조식도,  , 없으며,,  , 일반,  , 입주민들...
Name: output_words, dtype: object

In [10]:
post_sentences_df['post_words'].head()

0                          [너무너무,  , 만족스러운,  , 호텔이에요.]
1         [부산에,  , 오면,  , 꼭,  , 추천하고,  , 싶은,  , 곳이에요.]
2                                      [최고입니다!,  , ㅎㅎ]
3                     [다음에,  , 또,  , 올,  , 것,  , 같아요.]
4    [프론트가,  , 없고,,  , 조식도,  , 없으며,,  , 일반,  , 입주민들...
Name: post_words, dtype: object

In [11]:
for input_words, output_words, post_words in zip(sub_sentences['input_words'], sub_sentences['output_words'], post_sentences_df['post_words']):
    for i, (input_word, output_word, post_word) in enumerate(zip(input_words, output_words, post_words)):
        if len(input_word) != len(output_word): # 재복원하였음에도 글자수가 일치하지 않는 부분을 추출
            output_words[i] = post_word # 1차 후처리된 문장에 있는 단어들로 대체

In [12]:
# 전체 문장의 길이가 안맞는 경우에는 기존 후처리 방법을 통해 처리

outputs = []
for i, (input_text, output_words) in enumerate(zip(sub_sentences['input'], sub_sentences['output_words'])):
    new_output = ''.join(output_words)
    if len(new_output) < len(input_text):
        new_output += input_text[len(new_output):]
    elif len(new_output) > len(input_text):
        new_output = new_output[:len(input_text)]

    outputs.append(new_output)

In [13]:
sub_sentences['output'] = outputs

### 🏷️ 기존 난독화된 리뷰에 대응되는 부분 덮어쓰기

In [14]:
test_df = pd.read_csv('../data/test.csv', encoding='utf-8-sig') # 난독화된 리뷰

In [15]:
test_df.head()

,ID,input
0,TEST_0000,녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜...
1,TEST_0001,"풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰..."
2,TEST_0002,쥔차 붉찐졀행욘. 삶먼섶 멂묽럿턴 혹텔 중웨 쬐약위였습뉜따. 칙어뉜쥐 샤쨩윈쥐 쩨끄...
3,TEST_0003,붊 맛짚~~ 글련뎨 방움잃 뮈흙퍄녜용. 충칸 쏘움광 팔쿄닛갸 잊중짱임 야뉘럇셧 팜몌...
4,TEST_0004,빻 샹택는 쥔쨔 폐헐 칙젓뉜테 쩐맣은 죠하욧. 뽀읾럭카 알쥬 찬쟌합꿰 똘앝썬 츄어서...


In [16]:
output_texts = []
for idx, row in test_df.iterrows():
    input_text = row['input']
    mapping_row = sub_sentences[sub_sentences["ID"] == row["ID"]] # 난독화된 리뷰와 동일한 ID를 가지는 문장 단위의 복원된 리뷰들을 추출
    for _, mapping_row in mapping_row.iterrows(): # 복원된 문장들을 하나씩 대체
        output = mapping_row["output"]
        start_idx = mapping_row["start_idx"]
        end_idx = mapping_row["end_idx"]
        text = input_text[start_idx:end_idx]
        input_text = input_text.replace(text, output)
    output_texts.append(input_text) # 합쳐진 문장들을 저장


### 🏷️ 제출 파일에 저장하기

In [17]:
submission = pd.DataFrame({'ID': test_df['ID'], 'output': output_texts})
submission.to_csv('../submissions/submission_final.csv', index=False, encoding='utf-8-sig')